In [1]:
import pandas as pd
import yfinance as yf
import smtplib
from email.mime.text import MIMEText
import os
from dotenv import load_dotenv
import requests

In [2]:
# Load CSV file (replace 'stocks.csv' with your file name)
df = pd.concat([pd.read_csv(file) for file in ["QQQ.csv", "S&P.csv"]], ignore_index=True).drop_duplicates()
stock_symbols = df["Ticker"].tolist()

load_dotenv()

# Retrieve credentials from .env
DISCORD_WEBHOOK_URL = os.getenv("DISCORD_WEBHOOK_URL")

def get_stock_data(stock):
    data = yf.Ticker(stock).history(period="9mo")
    
    if data.empty:
        print(f"⚠ No data for {stock}, skipping...")
        return None, None  
    
    data["MA_150"] = data["Close"].rolling(window=150).mean()
    
    latest_price = data["Close"].iloc[-1]
    latest_ma_150 = data["MA_150"].iloc[-1]

    return latest_price, latest_ma_150

def is_near_ma_150(stock):
    try:
        price, ma_150 = get_stock_data(stock)
        
        if price is None or ma_150 is None:
            return None  

        # Define threshold for closeness (2%)
        lower_bound = ma_150 * 0.98
        upper_bound = ma_150 * 1.02

        if lower_bound <= price <= upper_bound:
            return f"🔔 {stock} ALERT: Price ${price:.2f} is near MA 150 (${ma_150:.2f})"

    except Exception as e:
        print(str(e))

    return None 

def send_discord_alert(message):
    """Send a Discord notification when stock is near MA 150"""
    webhook_url = DISCORD_WEBHOOK_URL
    data = {
        "content": message
    }

    try:
        response = requests.post(webhook_url, json=data)
        if response.status_code == 204:
            print("📲 Discord alert sent successfully!")
        else:
            print(f"⚠ Error sending Discord alert: {response.status_code}")
    except Exception as e:
        print(f"⚠ Error sending Discord alert: {e}")


In [3]:
for stock in stock_symbols:
    alert_message = is_near_ma_150(stock)
    if alert_message:
        print(alert_message)
        send_discord_alert(alert_message)

No significant breakout.
📲 Discord alert sent successfully!
No significant breakout.
📲 Discord alert sent successfully!
No significant breakout.
📲 Discord alert sent successfully!
No significant breakout.
📲 Discord alert sent successfully!
🔔 AVGO ALERT: Price $190.54 is near MA 150 ($190.02)
📲 Discord alert sent successfully!
No significant breakout.
📲 Discord alert sent successfully!
🔔 META ALERT: Price $586.00 is near MA 150 ($596.84)
📲 Discord alert sent successfully!
No significant breakout.
📲 Discord alert sent successfully!
No significant breakout.
📲 Discord alert sent successfully!
No significant breakout.
📲 Discord alert sent successfully!
No significant breakout.
📲 Discord alert sent successfully!
No significant breakout.
📲 Discord alert sent successfully!
No significant breakout.
📲 Discord alert sent successfully!
No significant breakout.
📲 Discord alert sent successfully!
No significant breakout.
📲 Discord alert sent successfully!
🔔 LIN ALERT: Price $457.75 is near MA 150 ($

$BRK.B: possibly delisted; no price data found  (period=9mo) (Yahoo error = "No data found, symbol may be delisted")


⚠ No data for BRK.B, skipping...


$BRK.B: possibly delisted; no price data found  (period=6mo) (Yahoo error = "No data found, symbol may be delisted")


IndexError: single positional indexer is out-of-bounds